In [2]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()
# Add src folder to path
project_root = Path.cwd().parent  # assumes notebook is in child folder
src_path = project_root / "sources"
sys.path.append(str(src_path))
from core.llm import OpenRouterClient
from assets.symbols import SYMBOL_SETS_JSON

/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
import json
from typing import List, Dict


def decode_double_encoded_utf8(s: str) -> str:
    """
    Converts strings like "\\u00e3\\u0082\\u00b1" back to "ケ".
    """
    try:
        # Step 1: Convert the string to bytes via 'latin-1'
        byte_like = s.encode("latin-1")
        # Step 2: Decode as UTF-8
        return byte_like.decode("utf-8")
    except Exception:
        return s  # fallback if decoding fails
    

def prompt_all_glyph_sets(
    glyphs_dict,
    llm,
    operation_types: List[str]
) -> List[Dict]:
    """
    Loops through all glyph sets in SYMBOL_SETS_JSON and asks an LLM to assign 
    likely transformation meanings to each symbol.
    
    Args:
        llm: An initialized LLM object (e.g., OpenRouterClient or ChatOpenAI instance)
        operation_types: List of allowed operation names
    
    Returns:
        A list of dicts with:
            - id
            - description
            - mapping: {symbol: inferred_operation}
    """
    results = []

    # Loop over all glyph sets in the dictionary
    for category in glyphs_dict.keys():
        print(f"Processing category: {category}")
        glyph_sets = glyphs_dict[category]

        for glyph_set in glyph_sets:
            try:
                # Validate structure
                if not isinstance(glyph_set, dict):
                    raise ValueError("Each glyph set must be a dictionary")

                glyph_id = glyph_set.get("id")
                description = glyph_set.get("description", "")
                symbols = glyph_set.get("symbols", [])

                if not glyph_id or not symbols:
                    raise ValueError(f"Invalid glyph set: missing 'id' or 'symbols' in {glyph_id}")

                # Decode any Unicode escape sequences in symbols
                decoded_symbols = [
                    decode_double_encoded_utf8(symbol) if isinstance(symbol, str) else symbol
                    for symbol in symbols
                ]

                prompt = f"""
                You are an expert in visual language and symbolic representation.

                You will be given a set of abstract symbols. Based on their appearance alone, your task is to categorize each symbol.

                Choose if a symbol suggests:
                - **sigil**: an action or movement that describes a transformation within a visual puzzle (e.g., a symbol that implies growth, rotation, or a change of state).
                - **gridglyph**: a static element designed to fit as a component on a grid (e.g., a marker, a fixed label, or a boundary element within a structured layout).

                Symbols: {', '.join(symbols)}

                For each symbol, respond with a single word (either "sigil" or "gridglyph") in the format:
                "symbol: description"

                Examples:
                - "A spiraling arrow: sigil"
                - "A solid square: gridglyph"
                - "An expanding circle: sigil"
                - "A dotted line: gridglyph"
                """

                # Call LLM
                response = llm(prompt).strip()
                symbol_to_op = response
                # Parse LLM response into symbol → operation mapping
                # symbol_to_op = {}
                # for line in response.splitlines():
                #     if ":" in line:
                #         symbol, op = line.split(":", 1)
                #         symbol = symbol.strip()
                #         op = op.strip()
                #         if op in operation_types:
                #             symbol_to_op[symbol] = op  # Already using real symbols here

                # Append result
                results.append({
                    "id": glyph_id,
                    "description": description,
                    "mapping": symbol_to_op
                })

            except Exception as e:
                print(f"Error processing glyph set '{glyph_id}': {str(e)}")
                continue

    return results

In [4]:
api_key = os.getenv("OPENROUTER_API_KEY")


llm = OpenRouterClient(
    api_key=api_key,
    model=r"qwen/qwen2.5-vl-72b-instruct",
    infra="deepinfra",
    max_tokens=1200,
    proxy=None,
    temperature=0.0,
    top_p=0.1,
    top_k=1,
    repetition_penalty=1.0,
    stream=False,
    seed=None
)

In [5]:
operation_types = OPERATION_TYPES = [
    "shift_right", "shift_left", "shift_up", "shift_down",
    "flip_h", "flip_v", "swap_rows", "swap_columns",
    "repeat_grid", "alternate", "sequence", "mirror_row", "mirror_column"
]

In [6]:
sample_glyph_set = SYMBOL_SETS_JSON

In [7]:
result = prompt_all_glyph_sets(sample_glyph_set, llm, operation_types)

print(json.dumps(result, indent=2, ensure_ascii=False))

Processing category: grid_glyphs
Processing category: operation_glyphs
[
  {
    "id": "katakana",
    "description": "Clean Japanese syllabary; monospaced, structured, and ideal for pattern recognition.",
    "mapping": "ツ: Bird\nレ: Wave\nハ: Breath\nア: Open\nヤ: Arrow\nユ: Circle\nヨ: Echo\nキ: Key\nク: Claw\nケ: Shell"
  },
  {
    "id": "emoji",
    "description": "Visually rich and distinct; great for attention-grabbing patterns but may distract in complex logic.",
    "mapping": "💥: Explosion  \n⚡: Lightning  \n🟥: Red square  \n🟦: Blue square  \n🌀: Whirlwind  \n✦: Sparkle  \n⚠️: Warning  \n✅: Checkmark  \n✈: Airplane  \n🛞: Gearwheel"
  },
  {
    "id": "runic",
    "description": "Ancient Germanic script with geometric clarity; excellent for mirroring, tiling, and symmetry tasks.",
    "mapping": "ᚠ: Beginning\nᚢ: Unity\nᚣ: Stability\nᚤ: Ascent\nᚥ: Connection\nᚦ: Transformation\nᚨ: Revelation\nᛇ: Harmony\nᛈ: Journey\nᛉ: Wisdom"
  },
  {
    "id": "box_drawing",
    "description": "Struc

In [8]:
print(json.dumps(result, indent=2, ensure_ascii=False))

[
  {
    "id": "katakana",
    "description": "Clean Japanese syllabary; monospaced, structured, and ideal for pattern recognition.",
    "mapping": "ツ: Bird\nレ: Wave\nハ: Breath\nア: Open\nヤ: Arrow\nユ: Circle\nヨ: Echo\nキ: Key\nク: Claw\nケ: Shell"
  },
  {
    "id": "emoji",
    "description": "Visually rich and distinct; great for attention-grabbing patterns but may distract in complex logic.",
    "mapping": "💥: Explosion  \n⚡: Lightning  \n🟥: Red square  \n🟦: Blue square  \n🌀: Whirlwind  \n✦: Sparkle  \n⚠️: Warning  \n✅: Checkmark  \n✈: Airplane  \n🛞: Gearwheel"
  },
  {
    "id": "runic",
    "description": "Ancient Germanic script with geometric clarity; excellent for mirroring, tiling, and symmetry tasks.",
    "mapping": "ᚠ: Beginning\nᚢ: Unity\nᚣ: Stability\nᚤ: Ascent\nᚥ: Connection\nᚦ: Transformation\nᚨ: Revelation\nᛇ: Harmony\nᛈ: Journey\nᛉ: Wisdom"
  },
  {
    "id": "box_drawing",
    "description": "Structured characters used for drawing boxes; ideal for spatial transformati

In [9]:
a = llm("""
You are given several pairs of 5x5 grids. Each pair shows how an input grid transforms into an output grid.

Study the pattern carefully and describe the logical transformation in plain English. Then, apply that same transformation to the final input grid and write out the resulting output grid.

---


Input (flattened):
['イ', 'ロ', 'ハ', 'ニ', 'ホ', 'ヘ', 'ト', 'チ', 'リ', 'ヌ', 'ス', 'セ', 'ソ', 'タ', 'カ', 'キ', 'ク', 'ケ', 'コ', 'サ', 'マ', 'ミ', 'ム', 'メ', 'モ']

Output (flattened):
['イ', 'ロ', 'ハ', 'ニ', 'ホ', 'ロ', 'ハ', 'ニ', 'ホ', 'ヘ', 'ハ', 'ニ', 'ホ', 'ヘ', 'ト', 'ニ', 'ホ', 'ヘ', 'ト', 'チ', 'ホ', 'ヘ', 'ト', 'チ', 'リ']

Apply the same transformation to this input:

Test Input:
[
    ['ア', 'イ', 'ウ', 'エ', 'オ'],
    ['カ', 'キ', 'ク', 'ケ', 'コ'],
    ['サ', 'シ', 'ス', 'セ', 'ソ'],
    ['タ', 'チ', 'ツ', 'テ', 'ト'],
    ['ナ', 'ニ', 'ヌ', 'ネ', 'ノ']
]

Describe the transformation first, then write the transformed output.
                """)

In [10]:
print(a)

### Transformation Description

The transformation rule observed between the input and output grids can be described as follows:

1. **Row-wise Shift**: For each row in the grid, starting from the second row, shift the characters one position to the left. The character that gets shifted out of the row on the left side is moved to the end of the row on the right side.

Let's break it down step-by-step for clarity:

- **First Row**: The first row remains unchanged.
- **Second Row**: The second row shifts all characters one position to the left, with the leftmost character moving to the rightmost position.
- **Third Row**: The third row also shifts all characters one position to the left, with the leftmost character moving to the rightmost position.
- **Fourth Row**: The fourth row follows the same pattern.
- **Fifth Row**: The fifth row follows the same pattern.

### Applying the Transformation

Now, let's apply this transformation to the provided test input grid:

**Test Input Grid:**
`

In [11]:
glyph = "\u1310"
print(glyph)

ጐ


In [12]:
print(llm(
prompt = f"""
Below are several examples of input-output pairs. Each input is a list of chess pieces (`♙` = pawn, `♘` = knight), and each output is the result after some kind of transformation.

Your task is to:

1. **Identify the pattern or rule** used in the transformations.
2. **Apply that same rule** to the final input and provide the correct output.

Think carefully about what changes between the inputs and outputs — items may move, shift, rotate, wrap around, or change position based on direction or frequency.
be consitent the transformation is from the input to the output

Could be a combination of moves — try to apply one move at a time and score it, like a chess player evaluating possibilities.

If you're unsure at first, look at multiple examples and compare them to spot the logic.
i want clear explaination, how many pieces move, treat each pieces indidually
---

Here are the transformation examples:

1. ["♙", "♙", "♘", "♙", "♙", "♙"] ➡ ["♙", "♙", "♙", "♘", "♙", "♙"]
2. ["♘", "♙", "♙", "♙", "♙"] ➡ ["♙", "♘", "♙", "♙", "♙"]
3. ["♙", "♘", "♙", "♘", "♙"] ⬅ ["♘", "♙", "♘", "♙", "♙"]
4. ["♙", "♙", "♙", "♘"] ⬅⬅⬅ ["♘", "♙", "♙", "♙"]
5. ["♙", "♘", "♙", "♙", "♘"] 𓁘⬅ ["♘", "♙", "♙", "♘", "♙"]
6. ["♙", "♙", "♘", "♙", "♙", "♙", "♙"] 𓁘⬅ ?

---

### Task:

Based on the previous patterns, what should replace the `?` in the last line?

👉 Replace the `?` with the correct array in this format:

["♙", "♙", "♘", "♙", "♙", "♙", "♙"] 𓁘⬅ ["?", "?", "?", "?", "?", "?", "?"]
"""
))

To solve this problem, we need to identify the pattern or rule governing the transformations between the input and output arrays. Let's analyze each example step by step.

### Example Analysis

1. **Example 1:**  
   Input: ["♙", "♙", "♘", "♙", "♙", "♙"]  
   Output: ["♙", "♙", "♙", "♘", "♙", "♙"]  
   - The knight ("♘") moves one position to the right.

2. **Example 2:**  
   Input: ["♘", "♙", "♙", "♙", "♙"]  
   Output: ["♙", "♘", "♙", "♙", "♙"]  
   - The knight ("♘") moves one position to the right.

3. **Example 3:**  
   Input: ["♙", "♘", "♙", "♘", "♙"]  
   Output: ["♘", "♙", "♘", "♙", "♙"]  
   - The knight ("♘") at index 1 moves one position to the left.

4. **Example 4:**  
   Input: ["♙", "♙", "♙", "♘"]  
   Output: ["♘", "♙", "♙", "♙"]  
   - The knight ("♘") moves three positions to the left.

5. **Example 5:**  
   Input: ["♙", "♘", "♙", "♙", "♘"]  
   Output: ["♘", "♙", "♙", "♘", "♙"]  
   - The knight ("♘") at index 1 moves one position to the left, and the knight at in

In [13]:
print(llm(
prompt = f"""You are a puzzle interpreter that understands Egyptian Hieroglyphs as symbolic actions.

Each hieroglyph represents a transformation rule. Here are some examples:

Glyph: 𓁘 (\u13058) → "Shift Right"
Glyph: 𓁙 (\u13059) → "Shift Left"

Example 1:
Input: [ . . # . . . ]
Rule: 𓁘
Output: [ . . . # . . ]

Example 2:
Input: [ # . . . . ]
Rule: 𓁘
Output: [ . # . . . ]

Example 3:
Input: [ . # . # . ]
Rule: 𓁙
Output: [ # . # . . ]

Now solve this puzzle:
Input: [ . . # . . . . ]
Rules: 𓁘 𓁘
What is the output?
"""
))

To solve the puzzle, we need to apply the given rules sequentially to the input array. The rules provided are:

- 𓁘 (8): Shift Right
- 𓁘 (8): Shift Right

Let's start with the initial input and apply each rule step by step.

### Initial Input:
\[ [ . . # . . . . ] \]

### Step 1: Apply the first rule (Shift Right)
When we shift right, each element moves one position to the right, and the last element wraps around to the first position.

\[ [ . . # . . . . ] \rightarrow [ . . . # . . . ] \]

### Step 2: Apply the second rule (Shift Right)
Now we take the result from the first step and apply another shift right.

\[ [ . . . # . . . ] \rightarrow [ . . . . # . . ] \]

### Final Output:
After applying both shift right rules, the final output is:

\[ [ . . . . # . . ] \]


In [14]:
print(f"""

[ . . 🟦 . . . ]
Rule: ⇝
[ . . . 🟦 . . ] 

[ 🟦 . . . . ]
Rule: ⇝
[ . 🟦 . . . ] 

[ . 🟦 . 🟦 . ]
Rule: ⇜
[ 🟦 . 🟦 . . ] 

[ . . . 🟦 ]
Rule: ⇜⇜⇜
[ 🟦 . . . ] 

[ . 🟦 . . 🟦 ]
Rule: ⇜
[ 🟦 . . 🟦 . ] 

[ . . 🟦 . . . . ]
Rules: 𓁘 𓁘⇜
? 
"""

)



[ . . 🟦 . . . ]
Rule: ⇝
[ . . . 🟦 . . ] 

[ 🟦 . . . . ]
Rule: ⇝
[ . 🟦 . . . ] 

[ . 🟦 . 🟦 . ]
Rule: ⇜
[ 🟦 . 🟦 . . ] 

[ . . . 🟦 ]
Rule: ⇜⇜⇜
[ 🟦 . . . ] 

[ . 🟦 . . 🟦 ]
Rule: ⇜
[ 🟦 . . 🟦 . ] 

[ . . 🟦 . . . . ]
Rules: 𓁘 𓁘⇜
? 

